# 30 - Sorting

---

The concept of "sorting" is an interesting one to teach students about the design of algorithms. By studying different approaches to sorting, you can see how different perspectives can be taken on one and the same problem, and how different techniques compare to each other.

---

## Sorting of lists

This chapter focuses on sorting of Python lists. In particular, it discusses the sorting of a list of numbers, from low to high. The techniques discussed, however, are not limited to just the sorting of lists of numbers -- with a few small adaptations they can be used to sort anything.

You probably know that Python lists have a built-in `sort()` method, which, when applied to a list of numbers, does exactly what the chapter discusses. So why would I see the need to discuss methods of sorting? The reasons are threefold:

- While lists have a built-in `sort()` method, you might get into a situation where you have to sort something that is not a list, and you have to implement your own method. This chapter will show you how to do that.
- The design of different algorithms for sorting is very instructive to teach you how to design algorithms in general.
- Comparison of different sorting algorithms teaches you more about algorithmic complexity.

### Python lists implementation

To be able to understand discussions on the manipulation of Python lists, you need to know something about how Python has implemented lists.

Python has implemented lists as "arrays". An array is a sequential block of computer memory that is divided into chunks of equal size, in such a way that each chunk contains one array element. Because the array consists of one sequential block of memory, it is very speedy for the computer to locate an element with a particular index.

An example illustrates this. Suppose that I have an array of 10 elements, each element having a size of 8 bytes. The first element (the one with index 0) starts at memory address `62560000` (in decimal numbers). Since the array consists of a sequential block of memory, the second element (index 1) will be at memory address `62560008`, the third at memory address `62560016`, etc. To find the element with index `i`, the computer needs to take the memory address of the first element, and add to that `i` times the size of each chunk -- in this case: `62560000 + i * 8`. Since for a computer accessing a memory address of which you know the number is a very fast operation, an array is an effective way of implementing a list.

It is not all good news, however. Arrays are notoriously rigid when you need to manipulate them. Most programming languages which use arrays force you to specify at the start how big the array should be. Python does not do that, it allows you to vary the length of the list. How exactly this is implemented I don't know, but it might happen that when you make a list grow longer and longer, at some point Python needs to pick up the whole list and put it somewhere else in memory, as there is no more room for the list to grow. Alternatively, Python may have implemented a method whereby some parts of the list exist somewhere else in memory. Regardless, Python needs to solve some issues to allow lists of variable length.

Moreover, while, in general, it is cheap to add a new element to the end of an array (`append()`), it is expensive to insert a new element in the middle or at the start of an array (`insert()`). The reason is that inserting requires the programming language to move all elements with an index greater than or equal to the place where the new element is inserted, one place upwards in memory. You can imagine that that is implemented according to the following idea:

In [ ]:
# insert num into tlist at index i.
def my_insert( tlist, num, i ):
    if i < 0 or i > len( tlist ):
        return
    tlist.append( tlist[-1] ) # add the last element of tlist once more at the end
    for j in range( len( tlist )-1, i, -1 ): # move up every element at or after i one place in the array
        tlist[j] = tlist[j-1]
    tlist[i] = num
    
numlist = [0,1,2,3,4,5,6,7,8,9,10]
my_insert( numlist, 99, 5 )
print( numlist )

Such an implementation of inserting is `O(n)` with `n` being the length of the list (i.e., in the worst-case scenario a new element is inserted at the start of the list, which means that every element needs to be copied once). Thus, inserting is a pretty slow process.

If you want to have a data structure that is more efficient at inserting elements in a list, you can use a "linked list". This will be discussed in a later chapter.

### In-place algorithms

There are two different ways of approaching sorting algorithms. 

The first is to use two lists: the original list that is to be sorted, and a new list that is sorted. Elements from the original list are taken and removed, and inserted into the new list. Once the original list is empty, the new list is the sorted version.

The second way uses only the original list, and sorts it "in-place". Usually the approach is that the list is considered to be partly sorted, e.g., the front end of the list (with the lowest indices) is sorted, and the back end is still unsorted. Elements are moved from the back end to the front end, and put in their rightful place, until the unsorted part of the list is empty.

Of several of the sorting algorithms below I discuss both ways of implementing them. In practice, usually only the "in-place" version is used, but often the first way is a bit easier to understand.

---

## Selection sort

In the introduction to this course (a long time ago) I gave an example whereby you have to sort four cards in ascending order. I asked you to write out, in English, a series of steps which produce, at the end, a sorted pile of cards. The simplest procedure would process the pile multiple times, each time seeking the highest card (or the lowest, depending on the direction in which you want to sort), and putting it aside. This method of sorting, which is called "selection sort", works as follows: 

In [ ]:
def selection_sort( tlist ):
    newlist = []
    while len( tlist ) > 0:
        num = min( tlist )
        newlist.append( num )
        tlist.remove( num )
    return newlist
        
numlist = [8, 3, 5, 11, 5, 33, 2, 2, 0, 9, 12, 26, 1, 9, 18]
numlist = selection_sort( numlist )
print( numlist )

__Exercise__: What is the time complexity of `selection_sort()` in big-O notation?

__Answer__: `n` is the number of items in the list. `selection_sort()` contains a loop. Each time through the loop, one element is removed, until the list is empty. Therefore, the loop is cycled through `n` times. Each time through the loop, there will be a selection of the minimum value, an append, and a remove. 

- Selection of the minimum means that in the best implementation each item is checked once. So we can say that this is `n` steps. Naturally, every time through the loop the number of items to check is reduced by 1, so that actually, on average, selecting the minimum needs `0.5*n` steps, but for time complexity estimation multiplication with a constant can be ignored.
- Appending an item to the end of a list needs 1 step.
- Removing an item from a list is about as complex as inserting an item into a list; instead of moving a whole bunch of items up, it means moving a whole bunch of items down. In the worst case, the first item from the list is removed, so we can say that this takes `n` steps (again, on average `0.5*n` steps, but we can ignore the constant).

Therefore, `selection_sort()` needs about `n * (n + 1 + n)` steps in the worst case, i.e., it is `O(n`&sup2;`)` (it is also &Theta;`(n`&sup2;`)`). This is pretty bad for any sorting algorithm.

The in-place version of selection sort is as follows:

In [ ]:
# returns the index of the minimum number in a list, starting at index start
def index_of_minimum( tlist, start ):
    low = start
    for i in range( start+1, len( tlist ) ):
        if tlist[i] < tlist[low]:
            low = i
    return low

def in_place_selection_sort( tlist ):
    for i in range( len( tlist ) ):
        low = index_of_minimum( tlist, i ) # find the index of the lowest at or after index i
        tlist[i], tlist[low] = tlist[low], tlist[i] # swap the items
            
numlist = [8, 3, 5, 11, 5, 33, 2, 2, 0, 9, 12, 26, 1, 9, 18]
in_place_selection_sort( numlist )
print( numlist )

You can check that this implementation of the selection sort method is also `O(n`&sup2;`)`. However, in absolute terms it is a bit faster than the previous implementation (it needs about half the steps), and it has, of course, the advantage that it only needs the original list.

Note that I implemented `index_of_minimum()` as a separate function. I could have replaced this with a call to `tlist.index( min( tlist ) )`, but that call has as disadvantage that it first moves through the list to find the minimum, and then moves through the list once more to find the index of the minimum. This call, while making the code quite a bit shorter, would also make it run at half speed. While in general I do not care about that, at this point in your programming career it is good to become aware of such features of an algorithm.

---

## Insertion sort

Selection sort is quite dumb. Insertion sort, at first glance, seems an improvement. The basic definition of insertion sort uses two lists: the original list, and a new list which contains the sorted elements. The new list starts empty. The algorithm grabs items from the original list, removing them, and inserts them into the new list. Since the new list is sorted, it can reasonably efficiently determine where the new item needs to be inserted. 

The main improvement over selection sort is that insertion sort does not need to select a particular item from the list (e.g., the minimum). It can simply take a random item from the list. If it takes the last one, removing it from the list is cheap (since no elements on the list need to be moved). 

Here is an example implementation of insertion sort:

In [ ]:
def find_index( tlist, num ):
    i = 0
    while i < len( tlist ):
        if tlist[i] >= num:
            break
        i += 1
    return i

def insertion_sort( numlist ):
    newlist = []
    while len( numlist ) > 0:
        num = numlist.pop()
        i = find_index( newlist, num ) # returns the spot where num must be inserted
        if i >= len( newlist ):
            newlist.append( num )
        else:
            newlist.insert( i, num )
    return newlist

numlist = [8, 3, 5, 11, 5, 33, 2, 2, 0, 9, 12, 26, 1, 9, 18]
numlist = insertion_sort( numlist )
print( numlist )

Insertion sort is `O(n`&sup2;`)`. It processes `n` items. For each item, it has to pop it (1 step), find the index (on average about `(0.5)*n` steps for the worst-case scenario) and insert it (on average about `(0.5)*n` steps for the worst-case scenario). So it needs about `n * (1 + (0.5)*n + (0.5)*n)` steps, which means that it is `O(n`&sup2;`)`.

__Exercise__: You can make the `find_index()` function more efficient by implementing it as a binary search. That does not change the time complexity of insertion sort, though. Why not? (Hint: Which of the terms in the steps formula which I give above represents the call to `find_index()`? Replace it by `log(n)`, which is the time-complexity of binary search.)

The "in-place" version of insertion sort considers the start of the list the sorted part of the list, and the end of the list the unsorted part. At the start, the sorted part is empty, and the unsorted part is (therefore) the whole list. The items of the unsorted part are processed from front to back (i.e., from low index to high index). To find the place where the insertion must be made, the item that must be inserted gets "swapped" with items from the back of the sorted part to the front, until it is in the correct spot. For example, suppose that of the following list the first 4 items are already sorted, and the other 4 not yet:

     2  5  11  12  4  1  0  9
     
The 4 is the next item to be processed. So it gets first swapped with the 12:

     2  5  11  4  12  1  0  9

Then with the 11:

     2  5  4  11  12  1  0  9
     
Then with the 5:

     2  4  5  11  12  1  0  9

And then the algorithm sees that the 4 is now in the correct spot, so it stops swapping. Now the sorted part of the list is 5 items long, and the unsorted only 3.

This is an elegant way of implementing insertion sort, which needs very little code.


In [ ]:
def in_place_insertion_sort( numlist ):
    if len( numlist ) <= 1:
        return
    for i in range( len( numlist ) ):
        j = i
        while j > 0 and numlist[j-1] > numlist[j]:
            numlist[j-1], numlist[j] = numlist[j], numlist[j-1]
            j -= 1
            
numlist = [8, 3, 5, 11, 5, 33, 2, 2, 0, 9, 12, 26, 1, 9, 18]
in_place_insertion_sort( numlist )
print( numlist )

This is still insertion sort, so it is `O(n`&sup2;`)` (check it if you have any doubts). However, when you study the algorithm, you might notice something about its big-Omega.

__Exercise__: What is the big-Omega of `in_place_insertion_sort()`?

__Answer__: The best-case scenario is that the list is already sorted to begin with. In that case, no swaps will take place at all; each item is checked only once, and then no longer handled. This means this in-place version of insertion sort is &Omega;`(n)`.

Note that the previously discussed selection sort does not have this advantage, as it has to seek the minimum each time it processes a new item. So selection sort is &Omega;`(n`&sup2;`)`. 

I said before that computer scientists are usually not interested in big-Omega. However, in case you are working with a list for which you know that it is already mostly sorted, the in-place version of insertion sort works well. In fact, if you know that on the list, on average an item is `k` places away from the place where it ultimately should end up, it is actually `O(nk)`. 

---

## Bubble sort

Bubble sort is a reasonably elegant sorting algorithm that has a lot in common with the in-place version of insertion sort. Bubble sort is by definition an in-place algorithm. It cycles through the list, from front to back, swapping neighboring items if they are not in the correct order. It continues cycling through the list until no more swaps are made. This algorithm is short and elegant:

In [ ]:
def bubble_sort( tlist ):
    while True:
        swap = False
        for i in range( len( tlist )-1 ):
            if tlist[i] > tlist[i+1]:
                tlist[i], tlist[i+1] = tlist[i+1], tlist[i]
                swap = True
        if not swap:
            return
        
numlist = [8, 3, 5, 11, 5, 33, 2, 2, 0, 9, 12, 26, 1, 9, 18]
bubble_sort( numlist )
print( numlist )

__Exercise__: Determine big-O and big-Omega for bubble sort.

__Answer__: Each cycle through the list, the item that is at the end of the original list can only be moved one position to the front. So if the item that is at the end of the original list is actually the item that should end up at index 0, the list is cycled through `n` times. Since each cycle touches each item once, the algorithm is `O(n`&sup2;`)`. The best-case scenario is that the list is sorted to begin with. In that case, no swaps are made at all, and the list is only processed once. So the algorithm is &Omega;`(n)`.

The time complexity of bubble sort is the same as of insertion sort. Its implementation is too. A close analysis will show that insertion sort tends to have a slight computational advantage. For instance, if you have a list that is already completely sorted, except that the very last item of the list should actually be at the front, then insertion sort needs about `n` checks and `n` swaps, while bubble sort needs about `n`&sup2; checks and `n` swaps. However, bubble sort has a slight advantage in being a bit easier to understand and implement (though I guess that is subjective).

---

## Merge sort

By now you might be wondering if there are any sorting algorithms that do better than `O(n`&sup2;`)`. Yes, there are. One of the most elegant implementations of a sorting algorithm is merge sort, which I will discuss now. 

Merge sort is a recursive algorithm. If you still do not understand recursion, you best go back to the chapter on that topic and study it (again), so that you understand the discussion.

The idea behind merge sort is as follows.

Suppose that I have two lists that are sorted. I can merge these together to form one sorted list. I simply create a third, empty list, check the first items on the two lists, and then remove the lowest and place it in the third list. I then do the same with what are now the first items on the two lists, and I continue doing that until one of the two lists is empty. At that point, I append the remaining items of the non-empty list at the end of the third list. The third list now contains all the elements from the first two lists, sorted. Such a merge operation can be implemented as `O(mn)`, whereby `m` is the length of the first list, and `n` is the length of the second list (I show this implementation below).

So merging two sorted lists is easy and efficient.

The merge sort algorithm takes a list, splits it into two halves, then calls some sorting procedure on each of the halves to produce sorted versions of the halves, and then merges the two halves together to produce the sorted list.

That sounds nice, but what is that "some sorting procedure" supposed to be? With the development of merge sort we are *supposed* to write a sorting procedure, right? So what do we call to sort the two halves? The answer is simple: we use merge sort to sort the two halves!

Wait a moment... That's circular. Merge sort needs merge sort to sort a list. How is that going to work?

The trick is that every time merge sort calls merge sort to sort another list, that list is at most half the length of the original list! So merge sort gets called with smaller and smaller lists, until it gets called with a list that is of length 1. And a list of length 1 is already sorted!

I can imagine that this sounds confusing, so here is an example.

Merge sort gets called with the following list:

    5  2  11  12  4  1  0  9
    
It splits that list in two halves, and calls merge sort with each of these halves, i.e., with:

    5  2  11  12   and   4  1  0  9
    
Each of these calls, again, calls merge sort with halved lists, so with:

    5  2   and   11  12   and   4  1   and   0  9
    
Each of these calls, once more, calls merge sort with halved lists, so with:

    5   and   2   and   11   and   12   and   4   and   1   and   0   and   9
    
These lists all have length 1, so they are all sorted. So now they get merged: the 5 and the 2 get merged, the 11 and the 12 get merged, the 4 and the 1 get merged, and the 0 and the 9 get merged, so we have:

    2  5   and   11  12   and   1  4   and   0  9
    
Then the first two sorted sublists get merged and the second two sorted sublists get merged, so we have:

    2  5  11  12   and   0  1  4  9
    
Finally, these two sorted sublists get merged, and we have:

    0  1  2  4  5  9  11  12

And there you have it. The way I see it, merge sort implements a sorting procedure without doing any sorting -- it only merges.

In [ ]:
def merge( sublist1, sublist2 ):
    newlist = []
    i1, i2 = 0, 0
    while i1 < len( sublist1 ) and i2 < len( sublist2 ):
        if sublist1[i1] < sublist2[i2]:
            newlist.append( sublist1[i1] )
            i1 += 1
        else:
            newlist.append( sublist2[i2] )
            i2 += 1
    newlist.extend( sublist1[i1:] )
    newlist.extend( sublist2[i2:] )
    return newlist

def merge_sort( tlist ):
    if len( tlist ) <= 1:
        return tlist
    sublist1 = merge_sort( tlist[:len( tlist )//2] )
    sublist2 = merge_sort( tlist[len( tlist )//2:] )
    return merge( sublist1, sublist2 )
        
numlist = [8, 3, 5, 11, 5, 33, 2, 2, 0, 9, 12, 26, 1, 9, 18]
numlist = merge_sort( numlist )
print( numlist )

Merge sort is `O(n*log(n))`. You can envision the algorithm as a tree, that has the whole list at the root, two branches under the root, each ending in a node that contains one half of the list, underneath each node again two branches, ending in a node that contains a quarter of the list, etc. At each level of the tree, all items on the list are processed once. This means that the `n` items on the list are all processed about as often as the depth of the tree, which is about `log`<sub>2</sub>`(n)`. Therefore merge sort is `O(n*log(n))`.

Note that I deliberately made sure that in the `merge()` function I do not remove any items from the sublists. If I would do that, the algorithm would become more expensive, as removing items from a list is an expensive operation, as I have shown before. Instead, I just keep track of the items at the front of the sublists with their respective indices.

I see two big disadvantages of merge sort. First, it needs quite a bit of extra memory to store all these temporary lists. It can be implemented as an in-place algorithm, but that may hurt the time complexity. Second, it is recursive, which not only means that it needs a lot of (stack) memory to store data on function calls, but also tends to be slow and get into problems if the lists get too big. 

Practical merge sort implementations should attempt to use the least amount of stack space possible, and try to reduce the number of calls to merge, for instance by combining merge sort with insertion sort when the lists get small. This is actually how the list `sort()` method in Python is implemented: it uses Timsort, which is a complex combination of merge sort and insertion sort, with several optimizations to make it work well with big sets of real-world data.

---

## Quicksort

For a long time, quicksort was seen as the best way to implement a general sorting algorithm. Quicksort works as follows:

- It first selects one number from the list, which is called the "pivot".
- It then moves all items bigger than the pivot to the back of the list, and all items smaller than the pivot to the front. This is called the "partition operation", and leads to the pivot being in its final place.
- Then the quicksort procedure is recursively repeated for the partitions. The recursion ends when a partition is of size 1 or less.

Note that the choice of pivot is important for the efficiency of the algorithm. A common choice is to simply take the last element of the list, but depending on the data, a different choice might be apt.

The following is a very efficient in-place implementation of quicksort.

In [ ]:
# This function creates two partitions around the pivot.
# The partitions are created in a sublist of tlist, namely between index lo and hi.
# Pivot is selected as the last item.
# The function then makes sure that all elements higher than pivot are at the end.
# Finally, it puts pivot between the two partitions, and returns the position of pivot.
def partition( tlist, lo, hi ):
    pivot = tlist[hi]
    i = lo - 1    
    for j in range( lo, hi ):
        if tlist[j] < pivot:
            i += 1
            tlist[i], tlist[j] = tlist[j], tlist[i]
    if tlist[hi] < tlist[i+1]:
        tlist[i+1], tlist[hi] = tlist[hi], tlist[i+1]
    return i+1

# This recursive function applies quicksort to tlist, between index lo and index hi.
def qs( tlist, lo, hi ):
    if lo < hi:
        p = partition( tlist, lo, hi )
        qs( tlist, lo, p-1 )
        qs( tlist, p+1, hi )

def quicksort( tlist ):
    qs( tlist, 0, len( tlist )-1 )
        
numlist = [8, 3, 5, 11, 5, 33, 2, 2, 0, 9, 12, 26, 1, 9, 18]
quicksort( numlist )
print( numlist )

Quicksort is `O(n*log(n))`, i.e., the same as merge sort. However, it tends to be two to three times faster than merge sort. Moreover, as the algorithm above shows, it can be easily implemented as an in-place procedure.

A disadvantage of quicksort is that it is "unstable". This means that if two elements of the list are considered of equal value, then after sorting they might have switched places. This does not matter for lists of numbers, of course, but if you want to sort, for instance, a deck of cards according to card value, and the 5 of Clubs is earlier in the deck than the 5 of Diamonds, then you might want to keep them in that order. With quicksort, it is no saying what their final order will be.

Naturally, quicksort also shares merge sort's disadvantages that come with it being recursive. However, the in-place implementation at least is very light on memory and stack usage.

---

## Discussion of sorting algorithms

There are many more sorting algorithms in existence, but most are variations of the ones above. The one that is probably missing in the discussion is "heapsort", which I will bring up briefly in the chapter on trees. All these sorting algorithms are "comparison sorts", which means that they are based on comparisons between elements. The best time complexity for the algorithms I showed is `O(n*log(n))`, which held for merge sort and quicksort. It can be proved mathematically that this is, in fact, optimal: for comparison sorts no better big-O time complexity can be achieved. Naturally, that does not mean that there is no difference between the algorithms which achieve this optimal big-O. Other factors might influence their efficiency in terms of time and memory.

---

## What you learned

In this chapter, you learned about:

- Python list implementation
- Selection sort
- Insertion sort
- Bubble sort
- Merge sort
- Quicksort

---

## Exercises

### Exercise 30.1

Cycle sort works as follows:

- Go through the elements of the list.
- For each element:
    - Count how many items on the list are smaller than the element; this gives the index of the element.
    - If the element is already located at the correct index, do nothing.
    - Else: move the element to its correct place, taking the element that is at that place and process that element next.

Implement cycle sort. Also determine its time complexity in big-O notation. Can you imagine any advantage of cycle sort?

Note: Cycle sort has two things that you have to be aware of:

- It is best to make sure that you use an in-place implementation, so that you will always make the correct counts. This means that you best not use a separate variable to store the element that you are processing, but always swap the value that you found at the location where the element is supposed to be with the element that you are currently processing.
- If the list contains double elements, you can get into an endless loop, as you may constantly be trying to switch an element with itself. To solve this, you have two options:
    - You insert the element not at the position that is indicated, but if you see that there is already an element with that value there, you insert it after the sequence of all the elements with that same value. However, you should take into account that if during the process of moving up in the list, you get to the index that you are currently moving an element from, that the element is already at its correct position.
    - You simply choose to implement this algorithm for lists with only unique values. This simplifies the development of the algorithm quite a bit. I my answer I will show both implementations. 

In [ ]:
# Cycle sort.


### Exercise 30.2

Cocktail shaker sort is a bi-directional bubble sort. It first "bubbles" up, after which you can be assured that at least the last element of the list is in the right place. Then it bubbles down from next-to-last element of the list to the first, after which you can be assured that the first element of the list is also in the right place. Then it bubbles up again, now starting with the second element, and up to the penultimate element. Then down again, and it keeps going up and down in smaller and smaller sublists until no more swaps take place.

Implement cocktail shaker sort. Determine its time complexity in big-O notation. Does it have advantages over bubble sort?

In [ ]:
# Cocktail shaker sort.


---

End of Chapter 30. Version 1.0. 